In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
item_detail_list_input_path = './data/intergrated_data/item/detail_list.csv'
item_id_list_input_path = './data/intergrated_data/item/id_list.csv'
item_overall_list_input_path = './data/intergrated_data/item/overall_list.csv'
item_rating_list_input_path = './data/intergrated_data/rating/rating_list.csv'

In [3]:
if not os.path.exists('./data/datasets'):
    os.makedirs('./data/datasets')

if not os.path.exists('./data/datasets/rating'):
    os.makedirs('./data/datasets/rating')

if not os.path.exists('./data/datasets/item'):
    os.makedirs('./data/datasets/item')

In [4]:
item_detail_list_out_path = './data/datasets/item/detail_list.csv'
item_id_list_out_path = './data/datasets/item/id_list.csv'
item_overall_list_out_path = './data/datasets/item/overall_list.csv'
item_rating_list_out_path = './data/datasets/rating/rating_list.csv'

In [5]:
details = pd.read_csv(item_detail_list_input_path, sep=',', encoding='utf-8')
ids = pd.read_csv(item_id_list_input_path, sep=',', encoding='utf-8')
overalls = pd.read_csv(item_overall_list_input_path, sep=',', encoding='utf-8')
ratings = pd.read_csv(item_rating_list_input_path, sep=',', encoding='utf-8')

In [6]:
details.describe()

,id,price,discount,discount_rate,n_sold,rank
count,2.587900e+04,2.587900e+04,2.587900e+04,25879.000000,24020.000000,17477.000000
mean,7.148851e+07,8.148962e+05,1.018593e+05,10.678079,641.487261,27.175202
std,4.114165e+07,3.453367e+06,6.598851e+05,16.670703,7140.842852,25.605199
min,5.466500e+04,-1.000000e+00,-1.000000e+00,-1.000000,-1.000000,-1.000000
25%,3.933278e+07,7.920000e+04,0.000000e+00,0.000000,13.000000,6.000000
50%,7.385575e+07,1.700000e+05,0.000000e+00,0.000000,60.000000,18.000000
75%,1.073252e+08,3.980000e+05,4.000000e+04,20.000000,273.000000,41.000000
max,1.524759e+08,1.107000e+08,5.096000e+07,91.000000,807078.000000,100.000000


In [7]:
details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25879 entries, 0 to 25878
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             25879 non-null  int64  
 1   name           25879 non-null  object 
 2   category       25879 non-null  object 
 3   brand          23801 non-null  object 
 4   price          25879 non-null  int64  
 5   discount       25879 non-null  int64  
 6   discount_rate  25879 non-null  int64  
 7   mota           25877 non-null  object 
 8   image_url      25879 non-null  object 
 9   n_sold         24020 non-null  float64
 10  rank           17477 non-null  float64
 11  main_category  25879 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.4+ MB


In [8]:
details.drop_duplicates(keep='first', inplace=True)

In [9]:
details = details[details['price'] >= 0]

In [10]:
details = details[details['image_url'] != -1]

In [11]:
ids.describe()

,id
count,2.587900e+04
mean,7.148851e+07
std,4.114165e+07
min,5.466500e+04
25%,3.933278e+07
50%,7.385575e+07
75%,1.073252e+08
max,1.524759e+08


In [12]:
ids.drop_duplicates(keep='first', inplace=True)

In [13]:
ids

,id
0,112329645
1,2454355
2,2454359
3,2454353
4,98565212
...,...
25874,37687523
25875,48759452
25876,17912191
25877,3110709


In [14]:
overalls.describe()


,id,avg_rating,n_reviews,n_rate_5,n_rate_4,n_rate_3,n_rate_2,n_rate_1,rate_with_img
count,2.484300e+04,24843.000000,24843.000000,24843.000000,24843.000000,24843.000000,24843.000000,24843.000000,24843.00000
mean,7.214107e+07,4.356607,105.339492,81.642555,14.951012,3.833917,1.472407,3.438474,9.77382
std,4.104887e+07,0.978872,282.763834,232.276753,38.002054,9.953808,3.933397,8.723980,15.77385
min,5.466500e+04,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
25%,4.121525e+07,4.300000,6.000000,4.000000,1.000000,0.000000,0.000000,0.000000,1.00000
50%,7.436138e+07,4.600000,24.000000,18.000000,3.000000,1.000000,0.000000,1.000000,3.00000
75%,1.077522e+08,4.800000,88.000000,65.000000,13.000000,3.000000,1.000000,3.000000,11.00000
max,1.524759e+08,5.000000,7127.000000,6465.000000,1054.000000,218.000000,93.000000,223.000000,100.00000


In [15]:
overalls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24843 entries, 0 to 24842
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             24843 non-null  int64  
 1   avg_rating     24843 non-null  float64
 2   n_reviews      24843 non-null  int64  
 3   n_rate_5       24843 non-null  int64  
 4   n_rate_4       24843 non-null  int64  
 5   n_rate_3       24843 non-null  int64  
 6   n_rate_2       24843 non-null  int64  
 7   n_rate_1       24843 non-null  int64  
 8   rate_with_img  24843 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 1.7 MB


In [16]:
overalls.dropna(inplace=True)

In [17]:
overalls

,id,avg_rating,n_reviews,n_rate_5,n_rate_4,n_rate_3,n_rate_2,n_rate_1,rate_with_img
0,112329645,4.5,20,14,3,2,1,0,0
1,2454355,4.7,365,309,38,4,4,10,60
2,2454359,4.8,432,383,26,11,2,10,58
3,2454353,4.8,367,324,25,6,2,10,60
4,98565212,4.7,276,240,15,5,3,13,49
...,...,...,...,...,...,...,...,...,...
24838,37687523,0.0,0,0,0,0,0,0,-1
24839,48759452,0.0,0,0,0,0,0,0,-1
24840,17912191,0.0,0,0,0,0,0,0,-1
24841,3110709,0.0,0,0,0,0,0,0,-1


In [18]:
overalls.drop_duplicates(keep='first', inplace=True)

In [19]:
overalls = overalls[overalls['avg_rating'] >= 0]
overalls = overalls[overalls['n_reviews'] >= 0]

In [20]:
ratings.describe()

,user_id,item_id,rating,timestamp
count,8.758640e+05,8.758640e+05,875864.000000,8.758640e+05
mean,1.176461e+07,5.668078e+07,4.402978,1.621381e+09
std,7.422492e+06,3.759287e+07,1.140255,2.017001e+07
min,1.000000e+01,5.589700e+04,1.000000,1.396550e+09
25%,6.226480e+06,2.144683e+07,4.000000,1.613360e+09
50%,1.161247e+07,5.646102e+07,5.000000,1.627130e+09
75%,1.795633e+07,8.157968e+07,5.000000,1.635677e+09
max,2.620566e+07,1.505549e+08,5.000000,1.640531e+09


In [21]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875864 entries, 0 to 875863
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    875864 non-null  int64 
 1   item_id    875864 non-null  int64 
 2   rating     875864 non-null  int64 
 3   timestamp  875864 non-null  int64 
 4   comment    564395 non-null  object
dtypes: int64(4), object(1)
memory usage: 33.4+ MB


In [22]:
ratings.drop_duplicates(subset=['item_id','comment'], keep='first', inplace=True)

In [23]:
ratings.to_csv(item_rating_list_out_path, sep=',', encoding='utf-8', index=False)
details.to_csv(item_detail_list_out_path, sep=',', encoding='utf-8', index=False)
ids.to_csv(item_id_list_out_path, sep=',', encoding='utf-8', index=False)
overalls.to_csv(item_overall_list_out_path, sep=',', encoding='utf-8', index=False)